In [2]:
import vitaldb
import pyvital.arr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import signal
import scipy as sp
import pyvital.filters.abp_ppv as f
import matplotlib.pyplot as plt
import statistics
import random
# df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
# df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information
# df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # laboratory results
import numpy as np
import matplotlib.pylab as plt
import padasip as pa
from scipy.signal import find_peaks
from tsfresh.feature_extraction.feature_calculators import time_reversal_asymmetry_statistic
import math
import scipy
import scipy.integrate
from statsmodels.tsa.seasonal import STL
import pyvital
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh import extract_features
from statsmodels.tsa.seasonal import STL
import seaborn as sns
import pandas as pd
import pandas
import smogn
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import seaborn as sns
from skrvm import RVR
#import catboost as cb
from sklearn.tree import DecisionTreeRegressor
import xgboost
from xgboost import XGBRegressor
from scipy import signal
import biosppy
import neurokit2 as nk

In [3]:
data = pd.read_csv('data_SV_500_new.csv')
data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508
0,0,2185.0,91.0,6.5,71.0,76.7,82.0,1.0,169.2,26.8,...,69.199402,68.211899,68.211899,68.211899,68.211899,69.199402,72.161797,76.111603,79.073997,82.036301
1,1,2185.0,91.0,6.5,71.0,76.7,82.0,1.0,169.2,26.8,...,71.174301,71.174301,70.186897,70.186897,71.174301,75.124100,77.099098,81.048897,85.986198,89.935997
2,2,2185.0,91.0,6.5,71.0,76.7,82.0,1.0,169.2,26.8,...,68.211899,67.224503,67.224503,66.237000,66.237000,67.224503,69.199402,71.174301,75.124100,78.086502
3,3,2185.0,91.0,6.5,71.0,76.7,82.0,1.0,169.2,26.8,...,71.174301,69.199402,69.199402,69.199402,68.211899,68.211899,68.211899,69.199402,72.161797,75.124100
4,4,2185.0,91.0,6.5,71.0,76.7,82.0,1.0,169.2,26.8,...,69.199402,68.211899,67.224503,67.224503,67.224503,67.224503,66.237000,65.249603,65.249603,66.237000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26604,26604,4800.0,103.0,10.5,102.0,69.5,34.0,1.0,170.7,23.9,...,70.186897,70.186897,70.186897,71.174301,71.174301,71.174301,71.174301,70.186897,70.186897,70.186897
26605,26605,4800.0,103.0,10.5,102.0,69.5,34.0,1.0,170.7,23.9,...,79.073997,77.099098,75.124199,73.149200,71.174301,69.199402,68.211998,67.224503,66.237099,66.237099
26606,26606,4800.0,103.0,10.5,102.0,69.5,34.0,1.0,170.7,23.9,...,86.973602,93.885803,99.810600,102.773003,105.735001,107.709999,107.709999,107.709999,105.735001,103.760002
26607,26607,4800.0,103.0,10.5,102.0,69.5,34.0,1.0,170.7,23.9,...,63.274700,64.262100,64.262100,65.249603,65.249603,65.249603,66.237099,66.237099,66.237099,66.237099


In [4]:
data_raw = data.to_numpy()
data_raw.shape

(26609, 1510)

In [ ]:
extracted_feature = np.array([])
train_features = np.array([])
cardica_cycles = [10]
nested_samples = np.array([])
count = 0
for cc in cardica_cycles:
    for i in range(0,len(data_raw)): 
        print(i)
        if np.isnan(data_raw[i][10:]).any():
            print("something Happend 4")
            continue
        signals, info = nk.eda_process(data_raw[i][10:], sampling_rate=100)
        bp_signal = signals['EDA_Raw']
        
        res = STL(bp_signal, period=2).fit()
        valid = True
        if (bp_signal > 200).any():
            print("something Happend 1")
            valid = False
        elif(bp_signal < 30).any():
            print("something Happend 2")
            valid = False
        elif (np.abs(np.diff(bp_signal)) > 30).any():  # abrupt change -> noise
            print("something Happend 3")
            valid = False
        elif np.isnan(bp_signal).any():
            print("something Happend 4")
            valid = False
        elif (np.max(bp_signal)-np.min(bp_signal)>70).any():
            valid = False
        if valid == False:
            nested_samples = np.append(nested_samples, i)
            print("something Happend 111")
            continue
        
        onsets = np.nan_to_num(info['SCR_Onsets'], nan=1, posinf=1, neginf=1).astype(int)
        systolics = np.nan_to_num(info['SCR_Peaks'], nan=1, posinf=1, neginf=1).astype(int)
        notches = np.nan_to_num(info['SCR_Recovery'], nan=1, posinf=1, neginf=1).astype(int)
        valid = True
        if len(onsets) < cc:
            valid = False
        if len(systolics) < cc:
            valid = False
        if valid == False:
            nested_samples = np.append(nested_samples, i)
            #print("something Happend 2")
            continue
        # find the smallest cardiac cycle and use it as a window for search
        window = np.min(np.diff(onsets))
        # find the onsets in the raw data
        systolics_raw = np.array([])
        onsets_raw = np.array([])
        notches_raw = np.array([])
        for s in onsets[1:]:
            systolics_raw = np.append(systolics_raw, s+signals['EDA_Raw'][s:s+window].argmax())
        for s in onsets[1:]:
            onsets_raw = np.append(onsets_raw, s+signals['EDA_Raw'][s:s+20].argmin())
        for s in notches[1:]:
            notches_raw = np.append(notches_raw, s + signals['EDA_Raw'][s:s+20].argmin())
        systolics_raw = systolics_raw.astype(int)
        onsets_raw = onsets_raw.astype(int)
        notches_raw = notches_raw.astype(int)
        valid = True
        if len(onsets_raw) < cc:
            valid = False
        if len(systolics_raw) < cc:
            valid = False
        if valid == False:
            nested_samples = np.append(nested_samples, i)
            print("something Happend 3")
            continue
        bp_cycle = signals['EDA_Raw'][onsets_raw[-cc]:onsets_raw[-1]]
        systolics_raw = systolics_raw[-cc:-1]
        onsets_raw = onsets_raw[-cc:-1]
        notches_raw = notches_raw[-cc:-1]
        ave = np.average(np.diff(bp_cycle))
        valid = True
        if np.max(bp_cycle) - np.min(bp_cycle) < 30:
            valid = False
        # elif ave > 0.1:
        #     valid = False
        if valid == False:
            nested_samples = np.append(nested_samples, i)
            print("something Happend 4")
            continue
        
        SBP = bp_cycle[systolics_raw]
        DBP = bp_cycle[onsets_raw]
        Pulse_pressure = SBP.values - DBP.values
        MAP = DBP + (Pulse_pressure/3)
        T_cycle = np.diff(onsets_raw)/100
        T_SP = np.abs(np.subtract(onsets_raw, systolics_raw))/100
        T_SN = np.abs(np.subtract(systolics_raw, notches_raw))/100
        T_DP = np.abs(np.subtract(onsets_raw, notches_raw))/100
        HR = 60/T_cycle
        Windkessel = SBP.values - DBP.values
        Windkessel_RC = (MAP.values[1:]/T_cycle)*np.log(SBP.values[1:]/DBP.values[1:])
        Liljestrand_Zander = ((SBP.values-DBP.values)/(SBP.values+DBP.values))
        Herd = MAP.values-DBP.values
        Pressure_RMS = np.array([])
        Systolic_area = np.array([])
        Systolic_area_Kouchoukos_correction = np.array([])
        
        for n in range(0, len(onsets_raw)):
            if len(bp_cycle.loc[onsets_raw[n]:notches_raw[n]]) == 0:
                continue
            Pressure_RMS = np.append(Pressure_RMS,math.sqrt((1/(100*len(bp_cycle.loc[onsets_raw[n]:notches_raw[n]])))*scipy.integrate.simps((bp_cycle.loc[onsets_raw[n]:notches_raw[n]])**2, dx=1/100)))
            Systolic_area = np.append(Systolic_area, scipy.integrate.simps(bp_cycle.loc[onsets_raw[n]:notches_raw[n]], dx=1/100))
            Systolic_area_Kouchoukos_correction = np.append(Systolic_area_Kouchoukos_correction, scipy.integrate.simps((1+(T_SP[n]/T_DP[n]))*bp_cycle.loc[onsets_raw[n]:notches_raw[n]], dx=1/100))
        
        extracted_feature = np.append(extracted_feature, data_raw[i][1])
        extracted_feature = np.append(extracted_feature, data_raw[i][2])
        extracted_feature = np.append(extracted_feature, data_raw[i][3])
        extracted_feature = np.append(extracted_feature, data_raw[i][4])
        extracted_feature = np.append(extracted_feature, data_raw[i][5])
        extracted_feature = np.append(extracted_feature, data_raw[i][6])
        extracted_feature = np.append(extracted_feature, data_raw[i][7])
        extracted_feature = np.append(extracted_feature, data_raw[i][8])
        extracted_feature = np.append(extracted_feature, data_raw[i][9])

        extracted_feature = np.append(extracted_feature, HR)
        extracted_feature = np.append(extracted_feature, T_SP)
        extracted_feature = np.append(extracted_feature, T_SN)
        extracted_feature = np.append(extracted_feature, T_DP)
        extracted_feature = np.append(extracted_feature, T_cycle)
        extracted_feature = np.append(extracted_feature, T_cycle)
        extracted_feature = np.append(extracted_feature, SBP)
        extracted_feature = np.append(extracted_feature, DBP)
        extracted_feature = np.append(extracted_feature, Pulse_pressure)
        extracted_feature = np.append(extracted_feature, MAP)
        ############
        extracted_feature = np.append(extracted_feature, Windkessel)
        extracted_feature = np.append(extracted_feature, Windkessel_RC)
        extracted_feature = np.append(extracted_feature, Liljestrand_Zander)
        extracted_feature = np.append(extracted_feature, Herd)
        extracted_feature = np.append(extracted_feature, Pressure_RMS)
        extracted_feature = np.append(extracted_feature, Systolic_area)
        extracted_feature = np.append(extracted_feature, Systolic_area_Kouchoukos_correction)
        #############
        seasonalitje = res.seasonal
        trend_mean = np.mean(res.trend)
        trend_std = np.std(res.trend)
        seasonal_mean = np.mean(res.seasonal)
        seasonal_std = np.std(res.seasonal)
        BP_mean = np.mean(signals['EDA_Raw'])
        BP_std = np.std(signals['EDA_Raw'])
        
        extracted_feature = np.append(extracted_feature, trend_mean)
        extracted_feature = np.append(extracted_feature, trend_std)
        extracted_feature = np.append(extracted_feature, seasonal_mean)
        extracted_feature = np.append(extracted_feature, seasonal_std)
        extracted_feature = np.append(extracted_feature, BP_mean)
        extracted_feature = np.append(extracted_feature, BP_std)
        
        df = pd.DataFrame({'value': signals['EDA_Raw']})
        df['id'] = 1
        features = extract_features(df, column_id='id', column_sort=None, column_kind=None, column_value='value')
        extracted_feature = np.append(extracted_feature, features['value__variance_larger_than_standard_deviation'][1])
        extracted_feature = np.append(extracted_feature, features['value__abs_energy'][1])
        extracted_feature = np.append(extracted_feature, features['value__absolute_maximum'][1])
        extracted_feature = np.append(extracted_feature, features['value__agg_autocorrelation__f_agg_"mean"__maxlag_40'][1])
        extracted_feature = np.append(extracted_feature, features['value__absolute_sum_of_changes'][1])
        extracted_feature = np.append(extracted_feature, features['value__sum_values'][1])
        extracted_feature = np.append(extracted_feature, features['value__mean_abs_change'][1])
        extracted_feature = np.append(extracted_feature, features['value__mean_change'][1])
        extracted_feature = np.append(extracted_feature, features['value__mean_second_derivative_central'][1])
        extracted_feature = np.append(extracted_feature, features['value__median'][1])
        extracted_feature = np.append(extracted_feature, features['value__mean'][1])
        extracted_feature = np.append(extracted_feature, features['value__length'][1])
        extracted_feature = np.append(extracted_feature, features['value__standard_deviation'][1])
        extracted_feature = np.append(extracted_feature, features['value__variation_coefficient'][1])
        extracted_feature = np.append(extracted_feature, features['value__variance'][1])
        extracted_feature = np.append(extracted_feature, features['value__skewness'][1])
        extracted_feature = np.append(extracted_feature, features['value__root_mean_square'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_0'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_1'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_2'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_3'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_4'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_5'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_6'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_7'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_8'][1])
        extracted_feature = np.append(extracted_feature, features['value__autocorrelation__lag_9'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_0'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_1'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_2'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_3'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_4'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_5'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_6'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_7'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_8'][1])
        extracted_feature = np.append(extracted_feature, features['value__partial_autocorrelation__lag_9'][1])
        extracted_feature = np.append(extracted_feature, features['value__linear_trend__attr_"pvalue"'][1])
        extracted_feature = np.append(extracted_feature, features['value__linear_trend__attr_"rvalue"'][1])
        extracted_feature = np.append(extracted_feature, features['value__linear_trend__attr_"intercept"'][1])
        extracted_feature = np.append(extracted_feature, features['value__linear_trend__attr_"slope"'][1])
        extracted_feature = np.append(extracted_feature, features['value__linear_trend__attr_"stderr"'][1])
        extracted_feature = np.append(extracted_feature, features['value__fourier_entropy__bins_2'][1])
        extracted_feature = np.append(extracted_feature, features['value__fourier_entropy__bins_3'][1])
        extracted_feature = np.append(extracted_feature, features['value__fourier_entropy__bins_5'][1])
        extracted_feature = np.append(extracted_feature, features['value__fourier_entropy__bins_10'][1])
        extracted_feature = np.append(extracted_feature, features['value__fourier_entropy__bins_100'][1])
        if i == 0:
            train_features = extracted_feature
            extracted_feature = []
        else:
            train_features = np.vstack([train_features, extracted_feature])
            extracted_feature = []
        
    df = pd.DataFrame(train_features)

df.to_csv('features_vitaldb.csv', encoding = 'utf-8-sig') 



In [ ]:
label = ['Unnamed: 0','ID','HR', 'CO','SV','W', 'Age', 'Sex', 'H', 'BMI',
         'HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8',
         'T_SP1','T_SP2','T_SP3','T_SP4','T_SP5','T_SP6','T_SP7','T_SP8',
         'T_SN1','T_SN2','T_SN3','T_SN4','T_SN5','T_SN6','T_SN7','T_SN8',
         'T_DP1','T_DP2','T_DP3','T_DP4','T_DP5','T_DP6','T_DP7','T_DP8',
         'T_C1','T_C2','T_C3','T_C4','T_C5','T_C6','T_C7','T_C8','T_C9',
         'T_C11','T_C12','T_C13','T_C14','T_C15','T_C16','T_C17','T_C18','T_C19',
         'SBP1','SBP2','SBP3','SBP4','SBP5','SBP6','SBP7','SBP8','SBP9',
         'DBP1','DBP2','DBP3','DBP4','DBP5','DBP6','DBP7','DBP8','DBP9',
         'pp1','pp2','pp3','pp4','pp5','pp6','pp7','pp8','pp9',
         'MAP1','MAP2','MAP3','MAP4','MAP5','MAP6','MAP7','MAP8','MAP9',
         'wind1','wind2','wind3','wind4','wind5','wind6','wind7','wind8','wind9',
         'wind_RC1','wind_RC2','wind_RC3','wind_RC4','wind_RC5','wind_RC6','wind_RC7','wind_RC8','wind_RC9',
         'li_za1','li_za2','li_za3','li_za4','li_za5','li_za6','li_za7','li_za8','li_za9',
         'Herd1','Herd2','Herd3','Herd4','Herd5','Herd6','Herd7','Herd8','Herd9',
         'P_rms1','P_rms2','P_rms3','P_rms4','P_rms5','P_rms6','P_rms7','P_rms8','P_rms9',
         'sys_area1','sys_area2','sys_area3','sys_area4','sys_area5','sys_area6','sys_area7','sys_area8','sys_area9',
         'sys_area_kc1','sys_area_kc2','sys_area_kc3','sys_area_kc4','sys_area_kc5','sys_area_kc6','sys_area_kc7','sys_area_kc8','sys_area_kc9',
         'trend_m', 'trend_std', 'seasonal_m', 'seasonal_std', 'BP_mean', 'BP_std',
         'var_larger_std','abs_energy','abs_max', 'agg_acc', 'abs_sum_changes',
         'sum_value', 'm_abs_change','m_change', 'm_s_derivative_c','median', 'mean',
         'length', 'std', 'vari_coeff', 'var', 'skewness', 'rms', 'ac0', 'ac1',
         'ac2','ac3','ac4','ac5', 'ac6', 'ac7', 'ac8', 'ac9', 'pac0', 'pac1', 'pac2',
         'pac3', 'pac4', 'pac5', 'pac6', 'pac7', 'pac8', 'pac9', 'lt_p_value',
         'lt_r_value', 'lt_intercept', 'lt_slope', 'lt_stderr', 'f_entropy2', 'f_entropy3',
         'f_entropy5', 'f_entropy10','f_entropy100']

df.columns = label
df